In [3]:
import pandas as pd, numpy as np, matplotlib.pyplot as plt, json, math, random
from datetime import datetime as dt
from mpl_toolkits.basemap import Basemap
from IPython.display import IFrame
%matplotlib inline

In [ ]:
[5]:

# define map colors
land_color = '#f5f5f3'
water_color = '#cdd2d4'
coastline_color = '#f5f5f3'
border_color = '#bbbbbb'
meridian_color = '#eaeaea'
marker_fill_color = '#0033cc'
marker_edge_color = 'None'

